# Lezione 36 — Knowledge Mining

**Obiettivi della lezione**
- Chiarire cosa sia una knowledge base e perché serve in analisi dati.
- Mostrare come estrarre conoscenza da documenti non strutturati con una pipeline semplice.
- Evidenziare limiti pratici (rumore, ambiguità, contesto) e come mitigarli.
- Posizionare il tema nel percorso: ponte tra data analytics classico e applicazioni AI basate su testo.

## Teoria concettuale approfondita

**Concetto di knowledge base**
- Archivio strutturato di informazioni che consente interrogazioni coerenti nel tempo (FAQ, policy, manuali, log arricchiti).
- Differenza da un semplice repository di file: informazioni normalizzate, relazioni esplicite, versioning e controlli di qualità.

**Estrazione di conoscenza (knowledge mining)**
- Processo per trasformare dati non strutturati (testo, pdf) in rappresentazioni utilizzabili da analisti e sistemi downstream.
- Passi tipici: raccolta → pulizia → segmentazione → indicizzazione → arricchimento (es. entità, categorie) → retrieval/analisi.

**Pipeline documentale semplificata**
1. **Ingestione**: acquisire documenti da fonti eterogenee.
2. **Normalizzazione**: rimuovere rumore (boilerplate, stopword), gestire encoding.
3. **Indicizzazione**: creare strutture per recupero efficiente (inverted index, TF-IDF, embeddings). Qui usiamo TF-IDF per semplicità.
4. **Enrichment**: estrarre metadati (entità, date) o tag manuali.
5. **Query & Ranking**: calcolare similarità tra query e documenti, ordinare per rilevanza.

**Spiegazione matematica essenziale (TF-IDF)**
- TF (term frequency): $tf_{t,d} = \frac{\text{conteggio}(t,d)}{\text{numero tot. termini in }d}$.
- IDF (inverse document frequency): $idf_t = \log \frac{N}{1 + n_t}$ dove $N$ è il numero di documenti e $n_t$ il numero di documenti che contengono il termine.
- Punteggio documento-query: prodotto scalare tra vettori TF-IDF di query e documenti; più alto ⇒ maggiore rilevanza.

**Assunzioni implicite**
- Il significato di un termine è approssimato dalla frequenza (bag-of-words: perde l'ordine).
- Documenti indipendenti; contesto sintattico e semantico ridotto.

**Errori concettuali comuni**
- Confondere estrazione di conoscenza con semplice full-text search.
- Pensare che TF-IDF risolva ambiguità semantiche (es. "Apple" frutto vs azienda).
- Trascurare la qualità dell'ingestione: garbage-in, garbage-out.

**Esempi mentali**
- Manuale interno: cercare "procedura rimborso" restituisce documenti con termini specifici; TF-IDF aumenta il peso di termini distintivi rispetto a parole comuni.

## Schema mentale / mappa logica

- **Quando usare**: FAQ aziendali, assistenza clienti, audit documentale, ricerca rapida su policy interne.
- **Quando non usare**: dataset troppo piccolo (pochi documenti), richieste che richiedono ragionamento simbolico complesso o comprensione profonda del contesto.
- **Segnali pratici nei dati**: molto testo libero, ripetizioni, termini distintivi; presenza di rumore (footer, boilerplate) da filtrare; necessità di ranking di pertinenza.
- **Pattern operativo**: definire fonte → normalizzare → indicizzare → testare con query reali → misurare precisione/recall qualitative → iterare pulizia/stopword/tag.

## Notebook dimostrativo (pipeline minimale TF-IDF)
Useremo un piccolo corpus testuale simulato per vedere una pipeline di knowledge mining semplificata: ingestione, pulizia minima, indicizzazione TF-IDF, query e osservazione dei limiti.

In [ ]:
# Importiamo le librerie minime per il testo e la gestione dei dati
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Creiamo un piccolo corpus simulato con documenti eterogenei
corpus = pd.DataFrame(
    {
        "id": ["doc1", "doc2", "doc3", "doc4"],
        "testo": [
            "Policy rimborsi viaggi: inviare ricevuta e approvazione manager.",
            "Guida interna sicurezza: password lunghe, MFA obbligatorio, niente condivisione.",
            "Procedura onboarding: account email, laptop configurato, formazione sicurezza.",
            "FAQ clienti: tempi spedizione standard 3-5 giorni lavorativi, resi entro 30 giorni."
        ],
    }
)
corpus

In [ ]:
# Definiamo il vettorizzatore TF-IDF con stopword italiane per ridurre il rumore
vectorizer = TfidfVectorizer(stop_words="italian")

# Addestriamo l'indice sul corpus (fit) e trasformiamo i testi in vettori sparsi
matrix = vectorizer.fit_transform(corpus["testo"])

# Salviamo anche la lista dei termini per consultazione
feature_names = vectorizer.get_feature_names_out()
feature_names[:10]  # mostriamo i primi 10 termini ordinati alfabeticamente

In [ ]:
# Definiamo una funzione di ricerca che:
# 1) Trasforma la query in TF-IDF
# 2) Calcola la similarità coseno con i documenti
# 3) Restituisce una tabella ordinata per punteggio

def cerca(query: str, top_k: int = 3):
    query_vec = vectorizer.transform([query])  # trasformiamo la query
    scores = cosine_similarity(query_vec, matrix).flatten()  # similarità coseno
    risultati = pd.DataFrame({"id": corpus["id"], "punteggio": scores})
    return risultati.sort_values(by="punteggio", ascending=False).head(top_k)

cerca("procedura rimborso viaggio")

In [ ]:
# Mostriamo due query per osservare limiti pratici
queries = [
    "sicurezza password",  # dovrebbe trovare doc2 e doc3
    "spedizione veloce"    # termine "veloce" non presente → punteggi bassi
]

risultati_demo = {q: cerca(q, top_k=4) for q in queries}
risultati_demo

### Osservazioni sul risultato
- La query su "sicurezza password" restituisce doc2 e doc3 con punteggi più alti: i termini distintivi coincidono.
- La query "spedizione veloce" produce punteggi bassi: il termine "veloce" non compare. Questo mostra sensibilità al vocabolario.
- Rumore/ambiguità: se inserissimo documenti con la parola "security" in inglese, la query in italiano potrebbe non trovarli senza stemming/translation.
- Mitigazioni: dizionari controllati, sinonimi, normalizzazione linguistica, tag manuali, feedback degli utenti.

## Esercizi svolti (step-by-step)
Gli esercizi mirano a consolidare: creazione dell'indice, valutazione qualitativa delle query, mitigazione del rumore.

In [ ]:
# Esercizio 1: aggiungiamo un documento sul tema rimborsi e rivalutiamo una query
nuovo_doc = pd.DataFrame(
    {"id": ["doc5"], "testo": ["Istruzioni rimborso spese: compilare modulo, allegare fatture, inviare a finance."]}
)
corpus_esteso = pd.concat([corpus, nuovo_doc], ignore_index=True)

# Ricostruiamo l'indice TF-IDF sul corpus esteso
vectorizer_esteso = TfidfVectorizer(stop_words="italian")
matrix_estesa = vectorizer_esteso.fit_transform(corpus_esteso["testo"])

# Definiamo una funzione di ricerca sul corpus esteso

def cerca_esteso(query: str, top_k: int = 5):
    query_vec = vectorizer_esteso.transform([query])
    scores = cosine_similarity(query_vec, matrix_estesa).flatten()
    return pd.DataFrame({"id": corpus_esteso["id"], "punteggio": scores}).sort_values(
        by="punteggio", ascending=False
    ).head(top_k)

cerca_esteso("rimborso spese viaggio", top_k=5)

In [ ]:
# Esercizio 2: confrontiamo l'effetto di aggiungere stopword personalizzate
stopword_extra = ["obbligatorio", "standard", "giorni"]  # termini poco informativi nel nostro dominio
vectorizer_stop = TfidfVectorizer(stop_words=stopword_extra + ["italian"])
matrix_stop = vectorizer_stop.fit_transform(corpus["testo"])

# Ricerca con stopword personalizzate
query = "spedizione standard resi"
query_vec = vectorizer_stop.transform([query])
scores = cosine_similarity(query_vec, matrix_stop).flatten()
risultati_stop = pd.DataFrame({"id": corpus["id"], "punteggio": scores}).sort_values(
    by="punteggio", ascending=False
)
risultati_stop

In [ ]:
# Esercizio 3: gestiamo una ambiguità semplice con sinonimi manuali
# Implementiamo una funzione di espansione della query con sinonimi noti del dominio
sinonimi = {
    "spedizione": ["consegna", "delivery"],
    "rimborso": ["reso", "refund"],
}


def espandi_query(query: str) -> str:
    parole = query.split()
    espansa = parole.copy()
    for p in parole:
        if p in sinonimi:
            espansa.extend(sinonimi[p])
    return " ".join(espansa)

query_base = "spedizione veloce"
query_espansa = espandi_query(query_base)

risultati_espansi = cerca(query_espansa, top_k=4)
query_base, query_espansa, risultati_espansi

## Conclusione operativa
- Portarsi a casa: knowledge mining richiede pipeline disciplinata; TF-IDF è un punto di partenza per ranking basico.
- Errori da evitare: ignorare la pulizia, sovrastimare TF-IDF per concetti ambigui, non validare con query reali.
- Ponte verso la prossima lezione: introduzione concettuale al deep learning, confrontando approcci classici (bag-of-words) con modelli neurali per il testo.